In [ ]:
import mujoco
import numpy
import mediapy as media
import matplotlib.pyplot as plt
import math
import yaml

In [ ]:
xml='''
<mujoco>
    <option timestep=".001"/>
    <option><flag contact="enable"/></option>
    <option gravity="0 0 -9.81"><flag gravity="enable"/></option>
    <compiler angle="degree"/>
    <worldbody>
        <light pos="0 0 1"/>
        <body name="floor" pos="0 0 0">
          <geom pos="0 0 -.05" size="5 5 .05" type="plane" rgba="1 0.83 0.61 0.5"/>
        </body>
        <body name="A" pos="0 0 1" axisangle="0 1 0 0">
            <joint name="j1" type="hinge" axis="0 1 0" pos="0 0 0" stiffness="0" damping="10" limited="false" range="0 45"/>
            <geom type="box" size=".5 .05 .05" pos=".5 0 0" rgba="1 0 0 1"/>
            <inertial mass="1" diaginertia="1 1 1" pos=".5 0 0"/>
            <body name="B" pos="1 0 0" axisangle="0 1 0 0">
                <joint name="j2" type="hinge" axis="0 1 0" pos="0 0 0"
                stiffness="0" damping="10" limited="false" range="0 45"/>
                <geom type="box" size=".5 .05 .05" pos=".5 0 0" rgba="0 1 0 1"  mass="1"/>
            </body>        
        </body>
    </worldbody>
</mujoco>
'''

In [ ]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

In [ ]:
mujoco.mj_resetData(model,data)
# data.qpos[0:2] = [45*math.pi/180,45*math.pi/180]
# data.qvel[0:2] = [-3,0]
frames=[]

duration = 3
framerate =30

q = []
w = []
t = []
x = []

while data.time<duration:

    mujoco.mj_step(model,data)
    q.append(data.qpos.copy())
    w.append(data.qvel.copy())
    x.append(data.xpos.copy())
    t.append(data.time)
    if len(frames)<data.time*framerate:
    
        renderer.update_scene(data)
        #renderer.update_scene(data,"world")
        pixels = renderer.render()
        frames.append(pixels)
        
media.show_video(frames,fps = framerate)

In [ ]:
q = numpy.array(q)
w = numpy.array(w)
x = numpy.array(x)
t = numpy.array(t)

In [ ]:
artist = plt.plot(t,q)
plt.legend(artist,['q1','q2'])

In [ ]:
artist = plt.plot(t,w)
plt.legend(artist,['w1','w2'])

In [ ]:
x.shape

In [ ]:
plt.figure()
plt.plot(x[:,:,0],x[:,:,2])

In [ ]:
import yaml

data = {}
data['t']=t.tolist()
data['q']=q.tolist()

with open('data.yml','w') as f:
    yaml.dump(data,f)

with open('data.yml') as f:
    data = yaml.load(f,Loader=yaml.Loader)
    
q = numpy.array(data['q'])
t = numpy.array(data['t'])
    

In [ ]:
artist = plt.plot(t,q)
plt.legend(artist,['q1','q2'])